In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd
df = pd.read_csv("data.csv")

In [3]:
df["MK"][0]

'https://cdn.shopify.com/s/files/1/0551/4310/4592/files/0_1_b8ca2194-2c3a-4a5d-a149-11a7213fb211.png?v=1720088271'

In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from PIL import Image
import os
import base64

# Initialize GPT (vision-capable)
llm = ChatOpenAI(model="gpt-4o-mini")

def ensure_supported_format(image_path: str) -> str:
    """
    Ensures the image is in JPG or PNG format.
    Converts WEBP (or other formats) to JPG automatically.
    Returns the path to the safe file.
    """
    ext = os.path.splitext(image_path)[1].lower()
    if ext in [".jpg", ".jpeg", ".png"]:
        return image_path  # already safe format
    
    # Convert unsupported formats (e.g., .webp) to .jpg
    img = Image.open(image_path).convert("RGB")
    safe_path = image_path.rsplit(".", 1)[0] + ".jpg"
    img.save(safe_path, "JPEG")
    return safe_path

def image_to_base64(image_path: str) -> str:
    """Reads a local image and returns a base64-encoded data URL."""
    with open(image_path, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode("utf-8")
    ext = os.path.splitext(image_path)[1].lower().replace(".", "")
    if ext == "jpg":
        ext = "jpeg"  # correct MIME type
    return f"data:image/{ext};base64,{b64}"

from langchain_core.messages import HumanMessage

def describe_image_with_langchain(
    llm,
    image_path: str,
    detail_level: str = "very detailed",
    item: str = "quilt",
    local_img: bool = True
):
    """
    Describe an image using an LLM via LangChain multimodal input.
    """

    if local_img:
        # Ensure we have a supported format
        safe_image_path = ensure_supported_format(image_path)

        # Encode local file as base64 data URL
        data_url = image_to_base64(safe_image_path)
    else:
        data_url = image_path

    # Build multimodal input
    message = HumanMessage(
        content=[
            {
                "type": "text",
                "text": (
                    f"Describe the {item} in {detail_level} for Midjourney without command, "
                    f"mentioning all visible niche, objects, colors, context, vibe and actions. "
                    f"Return the design only, no special character such as *, -. "
                    f"Example: a stunning quilt bedding set features a vibrant tree of Life design "
                    f"that blends intricate stitching and vibrant colors to evoke a sense of nature's "
                    f"beauty and harmony."
                ),
            },
            {"type": "image_url", "image_url": {"url": data_url}},
        ]
    )

    # Call LLM
    response = llm.invoke([message])
    return response.content

def tagging_image_with_langchain(llm, image_path: str, local_img = True):

    if local_img:    
        # Ensure we have a supported format
        safe_image_path = ensure_supported_format(image_path)
        
        # Encode local file as base64 data URL
        data_url = image_to_base64(safe_image_path)
    else:
        data_url = image_path
    
    # Build multimodal input
    message = HumanMessage(
        content=[
            {"type": "text", "text": f"Describe this design in detailed tags, including: niche, color, vibe, product type, design elements, theme. Return the result in raw JSON format, without code fences"},
            {"type": "image_url", "image_url": {"url": data_url}}
        ]
    )
    
    # Call GPT
    response = llm.invoke([message])
    return response.content

description = describe_image_with_langchain(llm, "charmingdesign.webp")
tags = tagging_image_with_langchain(llm, "charmingdesign.webp")
print("Detailed description:\n", description)
print(tags)


Detailed description:
 The quilt features a charming winter wonderland theme, showcasing a series of adorable snowmen adorned in festive hats and scarves. The main fabric is a soft, serene shade of light blue, creating a calming backdrop that evokes a gentle winter sky. Each snowman is meticulously embroidered, displaying vibrant colors like bright orange for their cheerful noses, deep reds and blues in their cozy attire, and delicate details like snowflakes scattered throughout.

The snowmen have distinct personalities: one is waving joyfully while another holds a decorative twig, and a third beams with a friendly smile. Additionally, decorative elements such as sparkling blue ornaments and rich red flowers add layers of texture and whimsy. The quilt is bordered with intricate stitching that frames the design beautifully, enhancing the overall winter vibe. The combination of the vibrant colors, playful imagery, and soft fabric creates a cozy, inviting atmosphere, perfect for snuggling

In [33]:
tags

'```json\n{\n    "niche": ["holiday", "winter", "seasonal", "home decor"],\n    "color": ["blue", "red", "white"],\n    "vibe": ["cozy", "festive", "cheerful"],\n    "productType": ["bedding", "quilt", "pillow"]\n}\n```'

In [10]:
description = describe_image_with_langchain(llm, df["MK"][0], local_img = False)
print(description + extra_requirement)

The quilt design features a collection of stylized cats in a variety of colors and patterns. Each cat is uniquely depicted with bold, expressive eyes and simple, elegant whiskers. The cats are arranged in a playful, overlapping manner, creating a dynamic, lively composition.

The color palette includes vibrant hues such as orange, green, black, white, purple, yellow, and blue. Each cat is a different color with some featuring combinations of colors and patterns, such as stripes and spots. The background consists of muted, earthy tones in geometric shapes that complement the vividness of the cats.

The fabric texture shows intricate quilting patterns that enhance the visual appeal, such as spirals and waves, lending depth and dimension to the overall design. This quilt exudes a whimsical, cheerful atmosphere, perfect for adding a touch of fun and creativity to any room. Put the final design on the quilt.
